In [1]:
# 環境構築

!apt-get update
!apt-get install -y libsndfile1-dev
!pip install scipy
!pip install scikit-learn
!pip install librosa

/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/a

In [2]:
# clips 作成

SAMPLING_RATE = 44100 # 変更不可

clip_size = 81920 # 楽曲を再構築するパーツ1つあたりの大きさ
step_size = 20480 # clip をずらすときの大きさ
window_size = 10240 # CQT_CHROMA を取得するのに使用するサンプル数
hop_size = 640 # window をずらすときの大きさ

from scipy.io.wavfile import read, write
import glob
import os
import numpy as np
import librosa
import sys

if os.path.isfile("../data/out/clips4predict.wav"):
    print("../data/out/clips4predict.wav exists.")
    files = ["../data/out/clips4predict.wav"]
else:
    # clips.npy をもとにデータを作成する 
    if os.path.isfile("../data/arrays/clips4predict.npy"):
        print("loading ../data/arrays/clips4predict.npy ...")
        clips = np.load("../data/arrays/clips4predict.npy")
        print("creating ../data/out/clips4predict.wav ...")
        write("../data/out/clips4predict.wav", SAMPLING_RATE, clips.reshape((clips.shape[0] * clips.shape[1], )))
        files = ["../data/out/clips4predict.wav"]
        
    # clips,npy がないとき /data/wav44100 内の WAV ファイルを参照する
    else:
        files = glob.glob("../data/wav4predict/*.wav")
        #files = ["../data/wav44100/3DEmbodimentFromLines.wav"] # デバッグ用 
        
if len(files) == 0:
    print("WAV ファイルが見つかりませんでした。")
    sys.exit(1)
        
clips_filename = "../data/arrays/c" + str(clip_size) + "_s" + str(step_size) + "_f32_clips4predict"

if os.path.isfile(clips_filename + ".npy"):
    print("loading " + clips_filename + ".npy ...")
    clips = np.load(clips_filename + ".npy")
else:
    print("creating " + clips_filename + ".npy ...")
    raw_data_list = [librosa.load(file, sr=SAMPLING_RATE)[0] for file in files] # 左の音だけ使う

    clips = np.zeros((0, clip_size), dtype=np.float32)
    for raw_data in raw_data_list:
        tmp = [raw_data[i:i + clip_size] for i in range(0, len(raw_data) - clip_size, step_size)]
        clips = np.vstack((clips, np.array(tmp)))
    np.save(clips_filename, clips)

print("The clip array has " + str(clips.shape[0]) + " clips.")

loading ../data/arrays/c81920_s20480_f32_clips4predict.npy ...
The clip array has 41637 clips.


In [3]:
# cqt 作成

n_bins = 84

cqt_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqts4predict"

if os.path.isfile(cqt_filename + ".npy"):
    print("loading " + cqt_filename + ".npy ...")
    cqts = np.load(cqt_filename + ".npy")
else:
    print("creating " + cqt_filename + ".npy ...")
    num = 0
    for clip in clips:
        print("cqt progress: clip " + str(num + 1) + " / " + str(len(clips)))
        
        tmp_cqt = librosa.cqt(clip, sr=SAMPLING_RATE, hop_length=hop_size, n_bins=n_bins)
        tmp_cqt = tmp_cqt.reshape((1, tmp_cqt.shape[0], tmp_cqt.shape[1]))
        
        if num == 0:
            cqts = tmp_cqt
        else:
            cqts = np.vstack((cqts, tmp_cqt))
            
        num += 1

    np.save(cqt_filename, cqts)
    
print("The cqt array has " + str(cqts.shape[0]) + " cqts.")
print("cqts.shape: " + str(cqts.shape))
print("Type(cqts[0][0][0]): " + str(type(cqts[0][0][0])))
print("np.max(cqts[0][0]): " + str(np.max(cqts[0][0])))

loading ../data/arrays/c81920_s20480_f32_clips4predict_w10240_h640_cqts4predict.npy ...
The cqt array has 41637 cqts.
cqts.shape: (41637, 84, 129)
Type(cqts[0][0][0]): <class 'numpy.complex64'>
np.max(cqts[0][0]): (0.04157509+0.022495579j)


In [4]:
# cqt_choroma 作成

def Preprocessing(array):
    array = np.abs(array)
    array = np.log(array + 1)
    array = array / np.log(np.finfo(np.float32).max)
    array = array.T
    return array

cqt_chroma_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqt_chromas4predict"

if os.path.isfile(cqt_chroma_filename + ".npy"):
    print("loading " + cqt_chroma_filename + ".npy ...")
    cqt_chromas= np.load(cqt_chroma_filename + ".npy")
else:
    print("creating " + cqt_chroma_filename + ".npy ...")
    num = 0
    for cqt in cqts:
        print("cqt_chroma progress: clip " + str(num + 1) + " / " + str(len(cqts)))
        
        tmp_cqt_chroma = librosa.feature.chroma_cqt(C=cqt, sr=SAMPLING_RATE)
        tmp_cqt_chroma = Preprocessing(tmp_cqt_chroma)
        tmp_cqt_chroma = tmp_cqt_chroma.reshape((1, tmp_cqt_chroma.shape[0], tmp_cqt_chroma.shape[1]))
        
        if num == 0:
            cqt_chromas = tmp_cqt_chroma
        else:
            cqt_chromas = np.vstack((cqt_chromas, tmp_cqt_chroma))
            
        num += 1

    np.save(cqt_chroma_filename, cqt_chromas)
    
print("The cqt_chroma array has " + str(cqt_chromas.shape[0]) + " cqt_chromas.")
print("cqt_chromas.shape: " + str(cqt_chromas.shape)) # clip 番号、window 番号、 chroma 番号 になる
print("Type(cqt_chromas[0][0][0]): " + str(type(cqt_chromas[0][0][0])))
print("np.max(cqt_chromas[0][0]): " + str(np.max(cqt_chromas[0][0])))

loading ../data/arrays/c81920_s20480_f32_clips4predict_w10240_h640_cqt_chromas4predict.npy ...
The cqt_chroma array has 41637 cqt_chromas.
cqt_chromas.shape: (41637, 129, 12)
Type(cqt_chromas[0][0][0]): <class 'numpy.float32'>
np.max(cqt_chromas[0][0]): 0.0078125


In [5]:
# max_norm 作成

num_divide = 8

max_norm_filename = clips_filename + "_d" + str(num_divide) + "_max_norms4predict"

if os.path.isfile(max_norm_filename + ".npy"):
    print("loading " + max_norm_filename + ".npy ...")
    max_norms= np.load(max_norm_filename + ".npy")
else:
    print("creating " + max_norm_filename + ".npy ...")
    num = 0
    for clip in clips:
        print("max_norm progress: clip " + str(num + 1) + " / " + str(len(clips)))
        max_norm = [np.max(clip[int((i / num_divide) * len(clip)):int(((i + 1) / num_divide) * len(clip))] ** 2) for i in range(num_divide)]
    
        if num == 0:
            max_norms = np.array(max_norm)
        else:
            max_norms = np.vstack((max_norms, np.array(max_norm)))
        num += 1
        
    np.save(max_norm_filename, max_norms)
    
print("The max_norm array has " + str(max_norms.shape[0]) + " max_norms.")
print("max_norms.shape: " + str(max_norms.shape))
print("Type(max_norms[0][0]): " + str(type(max_norms[0][0])))
print("np.max(max_norms[0]): " + str(np.max(max_norms[0])))

loading ../data/arrays/c81920_s20480_f32_clips4predict_d8_max_norms4predict.npy ...
The max_norm array has 41637 max_norms.
max_norms.shape: (41637, 8)
Type(max_norms[0][0]): <class 'numpy.float32'>
np.max(max_norms[0]): 0.14306161


In [6]:
# 学習データとテストデータに分ける

def normalize(array):
    if (np.sum(array) == 0):
        return array
    else:
        return array / np.sum(array)
    
cqt_chroma_sum_threshold = 0.01 # 次の window の sum が閾値に満たないときに除外します
test_data_rate = 0.1

window_num_per_clip = cqt_chromas.shape[1]

cqt_chromas = np.array([np.hstack((cqt_chromas[i], np.repeat(np.array([max_norms[i]]), cqt_chromas.shape[1], axis=0))) for i in range(len(max_norms))])
cqt_chromas = np.concatenate([cqt_chromas[:-1, :, :], cqt_chromas[1:, 0, :].reshape(cqt_chromas.shape[0] - 1, 1, cqt_chromas.shape[2])], 1)

p = np.random.permutation(len(cqt_chromas))
cqt_chromas = cqt_chromas[p]
clips = clips[p]

x = np.delete(cqt_chromas, np.where(np.sum(cqt_chromas[:,-1,:], axis=1) < cqt_chroma_sum_threshold)[0], axis=0)

x_test = x[:int(x.shape[0] * test_data_rate), :window_num_per_clip, :].reshape(int(x.shape[0] * test_data_rate), window_num_per_clip, x.shape[2], 1)
x_train = x[int(x.shape[0] * test_data_rate):, :window_num_per_clip, :].reshape(x.shape[0] - int(x.shape[0] * test_data_rate), window_num_per_clip, x.shape[2], 1)

y = np.array([x[:, window_num_per_clip, i] for i in range(x.shape[2])])
y_maxs = np.max(y, axis=1)
y_mins = np.min(y, axis=1)

y_tests =  np.array([(x[:int(x.shape[0] * test_data_rate), window_num_per_clip, i] - y_mins[i]) / (y_maxs[i] - y_mins[i]) for i in range(x.shape[2])])
y_trains = np.array([(x[int(x.shape[0] * test_data_rate):, window_num_per_clip, i] - y_mins[i]) / (y_maxs[i] - y_mins[i]) for i in range(x.shape[2])])

print("x_train.shape: " + str(x_train.shape))
print("x_test.shape: " + str(x_test.shape))
print("y_trains.shape: " + str(y_trains.shape))
print("y_tests.shape: " + str(y_tests.shape))

x_train.shape: (37439, 129, 20, 1)
x_test.shape: (4159, 129, 20, 1)
y_trains.shape: (20, 37439)
y_tests.shape: (20, 4159)


In [12]:
# 楽曲の出力

import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import load_model
import soundfile as sf

epochs = 200
batch_size = 64
data_name = "c" + str(clip_size) + "_s" + str(step_size) + "_w" + str(window_size) + "_h" + str(hop_size) + "_d" + str(num_divide)

def add_fade(x, ft_len):
    r = np.arange(0, ft_len)*np.pi/ft_len
    w_fo = (0.5+0.5*np.cos(r))**0.5
    w_fi = (0.5-0.5*np.cos(r))**0.5
    
    x[0:ft_len]        *= w_fi
    x[clip_size-ft_len::] *= w_fo
    return x

def gen_xfade(x_pre, x_next, ft_len):
    if x_pre is None:
        xfade = x_next
    else:
        x_pre_begin = x_pre[:x_pre.shape[0] - ft_len]
        x_pre_end = x_pre[x_pre.shape[0] - ft_len:]
        x_pre_len = clip_size
        x_next_len = clip_size
        x_pre_len -= ft_len
        x_next_len -= ft_len
        xfade = np.concatenate((x_pre_begin, np.concatenate((x_pre_end, np.zeros(x_next_len))) + x_next))
    return xfade

def create_music_consider_weights(fname, rw):
    first_index = np.random.randint(0, len(cqt_chromas))
    predict_index = first_index
    out = None
    fade_samples = 10240
    
    for i in range(num_clips):
        print("-- generate " + str(i + 1) + " / " + str(num_clips))
        predict_vec = np.zeros((0, ))
        for i in range(x.shape[2]):
            predict = models[i].predict(np.array([cqt_chromas[predict_index, :-1]]))[0] * (y_maxs[i] - y_mins[i]) + y_mins[i]
            predict_vec = np.hstack((predict_vec, predict))
        predict_chroma = predict_vec[:len(predict_vec)-num_divide]
        predict_rhythm = predict_vec[len(predict_vec)-num_divide:]

        index = 0
        similar_index = 0
        score = -1 - rw
        chroma_score = 0
        rhythm_score = 0
        for cqt_chroma in cqt_chromas[:, (int)(fade_samples / hop_size)]:
            cqt_chroma_chroma = cqt_chroma[:len(cqt_chroma)-num_divide]
            cqt_chroma_rhythm = cqt_chroma[len(cqt_chroma)-num_divide:]
            tmp_chroma_score = cosine_similarity(np.array([predict_chroma]), np.array([cqt_chroma_chroma]))
            tmp_rhythm_score = cosine_similarity(np.array([predict_rhythm]), np.array([cqt_chroma_rhythm]))
            tmp_score = tmp_chroma_score + rw * tmp_rhythm_score
            if tmp_score > score:
                score = tmp_score
                similar_index = index
            index += 1

        print("score: " + str(score))
        print("predict_vec: " + str(predict_vec))
        print("cqt_chromas[similar_index]: " + str(cqt_chromas[similar_index, 0]))
        print("similar_index: " + str(similar_index))
        print("--")

        predict_index = similar_index

        tmp = add_fade(clips[predict_index], fade_samples)
        out = gen_xfade(out, tmp, fade_samples)
        
    sf.write(fname, out, SAMPLING_RATE, subtype="PCM_16")

models = []
for i in range(x.shape[2]):
    model_name = "independent_" + data_name + "_batch" + str(batch_size) + "_e" + str(epochs) + "_" + str(i)
    
    models.append(load_model("../data/models/" + model_name))

num_clips = 30

for i in range(21):
    rhythm_weight = 0.40 + i * 0.01
    file_name = "out_" + "independent_" + data_name + "_batch" + str(batch_size) + "_e" + str(epochs) + "_rw" + format(rhythm_weight, '.2f').replace('.', '_') + "_PREDICT"
    fname = "../data/out/" + file_name + "_track" + str(i) + ".wav"
    print("creating " + fname + " ...")
    create_music_consider_weights(fname, rhythm_weight)

creating ../data/out/out_independent_c81920_s20480_w10240_h640_d8_batch64_e200_rw0_40_PREDICT_track0.wav ...
-- generate 1 / 30
1/1 [==============================] - 0s 37ms/step
score: [[1.3089575]]
predict_vec: [0.00182263 0.0051593  0.00233222 0.00502441 0.00178302 0.00082044
 0.00134327 0.00066197 0.0003003  0.0005333  0.00541793 0.00135434
 0.00603737 0.00301286 0.01028603 0.01507772 0.00667631 0.00506776
 0.01429715 0.02327042]
cqt_chromas[similar_index]: [0.00525656 0.         0.         0.00572576 0.         0.00393037
 0.         0.0078125  0.         0.         0.         0.
 0.09653416 0.07896429 0.07722455 0.1207694  0.13725795 0.09776118
 0.16088746 0.1816927 ]
similar_index: 35882
--
-- generate 2 / 30
1/1 [==============================] - 0s 16ms/step
score: [[1.34157762]]
predict_vec: [0.00065275 0.00455814 0.00487599 0.00140126 0.00267667 0.00080496
 0.00166505 0.00365044 0.0009583  0.00062335 0.00024768 0.00191411
 0.04076649 0.06726024 0.0695694  0.05391653 0.08630

1/1 [==============================] - 0s 15ms/step
score: [[1.33835708]]
predict_vec: [0.00075108 0.00198859 0.00011132 0.0019723  0.00234027 0.00139277
 0.00136735 0.00504794 0.00171079 0.00120105 0.00200721 0.00739156
 0.02581152 0.027836   0.01841909 0.02715525 0.0372438  0.02507473
 0.02949895 0.03516022]
cqt_chromas[similar_index]: [0.         0.00274262 0.         0.         0.         0.00390293
 0.00582783 0.0078125  0.         0.         0.00137819 0.003063
 0.0730837  0.07802379 0.0581312  0.06269852 0.12483308 0.10462366
 0.08304626 0.07686032]
similar_index: 10024
--
-- generate 7 / 30
1/1 [==============================] - 0s 20ms/step
score: [[1.34117022]]
predict_vec: [0.00276877 0.00053025 0.00041679 0.00137901 0.00181418 0.00556457
 0.0013833  0.00284875 0.00093874 0.00207442 0.00666644 0.001589
 0.03095344 0.03619248 0.06223239 0.05775206 0.04498919 0.04277395
 0.04725474 0.04415162]
cqt_chromas[similar_index]: [0.         0.00487833 0.         0.00266436 0.         

1/1 [==============================] - 0s 16ms/step
score: [[1.342418]]
predict_vec: [0.00100254 0.0039822  0.00468254 0.00073942 0.00087547 0.00115402
 0.00348144 0.00060326 0.00020354 0.00107076 0.00155868 0.00398053
 0.0788927  0.0608555  0.06044589 0.10752894 0.07127009 0.0775475
 0.06476545 0.0707184 ]
cqt_chromas[similar_index]: [0.00688916 0.00313887 0.00247951 0.00424874 0.00375723 0.00513771
 0.0078125  0.         0.         0.         0.00294918 0.
 0.1338982  0.10652749 0.09168506 0.09036842 0.09872729 0.1254052
 0.14111753 0.16213845]
similar_index: 5117
--
-- generate 12 / 30
1/1 [==============================] - 0s 15ms/step
score: [[1.33557944]]
predict_vec: [0.00082072 0.00291568 0.00073024 0.002226   0.00337243 0.00211091
 0.00493642 0.00115753 0.0009524  0.00072926 0.0024795  0.00563471
 0.04985413 0.05071023 0.04987963 0.07045091 0.07654275 0.09158402
 0.08413094 0.07189894]
cqt_chromas[similar_index]: [0.         0.         0.         0.         0.         0.
 0.00

1/1 [==============================] - 0s 16ms/step
score: [[1.34243329]]
predict_vec: [0.00238463 0.00218787 0.00043224 0.00255948 0.00209503 0.00298544
 0.00234704 0.00697849 0.00067841 0.00079787 0.00153528 0.00030873
 0.06187543 0.05969088 0.03575979 0.06791599 0.04854058 0.0786704
 0.06438819 0.05600395]
cqt_chromas[similar_index]: [0.00131981 0.00257466 0.0029185  0.00102492 0.0027668  0.00269267
 0.0078125  0.         0.         0.         0.00136145 0.
 0.07991906 0.05080644 0.02895647 0.04994341 0.03637397 0.05513926
 0.0316492  0.03408873]
similar_index: 33800
--
-- generate 17 / 30
1/1 [==============================] - 0s 18ms/step
score: [[1.32974194]]
predict_vec: [0.00027894 0.00163142 0.00205958 0.00093983 0.00206288 0.00234474
 0.00513596 0.00467601 0.00130493 0.00227101 0.00014293 0.00020757
 0.01649571 0.02697638 0.01675916 0.02948741 0.01619809 0.01848787
 0.02453517 0.03284714]
cqt_chromas[similar_index]: [0.00297925 0.         0.         0.00183059 0.00324898 0.00

1/1 [==============================] - 0s 19ms/step
score: [[1.32329164]]
predict_vec: [0.00289176 0.00025615 0.00453561 0.00158255 0.001276   0.00097893
 0.00189502 0.00121096 0.00104118 0.00065695 0.00025902 0.00630035
 0.04823636 0.0793236  0.0491924  0.05807174 0.05802251 0.03066359
 0.03827091 0.04220415]
cqt_chromas[similar_index]: [0.         0.         0.0078125  0.         0.         0.
 0.00514615 0.         0.00593099 0.00235063 0.00488476 0.
 0.0902217  0.07947107 0.08931649 0.08790842 0.05451757 0.03739375
 0.0587805  0.06580803]
similar_index: 34277
--
-- generate 22 / 30
1/1 [==============================] - 0s 20ms/step
score: [[1.35541495]]
predict_vec: [0.00496414 0.00226343 0.00325028 0.00059889 0.0008733  0.00138393
 0.000714   0.00096449 0.00070757 0.00103698 0.0020858  0.00559394
 0.04834007 0.04973036 0.02734844 0.02125961 0.03061417 0.03800363
 0.03820755 0.03908306]
cqt_chromas[similar_index]: [0.         0.         0.         0.         0.0032757  0.
 0.     

...

1/1 [==============================] - 0s 16ms/step
score: [[1.54430546]]
predict_vec: [0.00483515 0.00527967 0.00077623 0.0054211  0.00434696 0.00090952
 0.00058521 0.00084381 0.00081979 0.0010662  0.00046725 0.00014402
 0.03335696 0.04294064 0.02994991 0.02468204 0.01925876 0.01421142
 0.02221217 0.03651373]
cqt_chromas[similar_index]: [0.00347283 0.         0.0078125  0.00663006 0.         0.
 0.00728854 0.00536998 0.0023739  0.         0.         0.00370988
 0.24833956 0.28466368 0.1840941  0.19392455 0.20088002 0.21549848
 0.15357894 0.3618958 ]
similar_index: 28390
--
-- generate 8 / 30
1/1 [==============================] - 0s 16ms/step
score: [[1.54038584]]
predict_vec: [9.12656658e-04 5.38493600e-03 4.88767074e-03 1.88255857e-04
 8.18276836e-04 1.66756508e-03 2.72869575e-03 1.06344686e-03
 4.20015684e-04 3.29180411e-03 3.06021800e-04 1.63413445e-03
 1.00510962e-01 1.06888279e-01 1.03365935e-01 1.21845432e-01
 8.24238509e-02 2.04698250e-01 1.57867655e-01 1.55766159e-01]
cqt_chr

1/1 [==============================] - 0s 17ms/step
score: [[1.48622077]]
predict_vec: [0.00110121 0.00081973 0.00389742 0.00390858 0.00080374 0.00290081
 0.00204479 0.00399609 0.00417526 0.00060052 0.00091349 0.00252943
 0.03358502 0.13434324 0.04795685 0.06358088 0.05668143 0.07849832
 0.06601922 0.05872702]
cqt_chromas[similar_index]: [0.00332206 0.00435196 0.00330915 0.00230269 0.0019538  0.
 0.00580638 0.0078125  0.00487292 0.         0.00106027 0.
 0.22924013 0.23982134 0.13717882 0.1491504  0.12743427 0.14193265
 0.2130546  0.2211307 ]
similar_index: 8767
--
-- generate 13 / 30
1/1 [==============================] - 0s 17ms/step
score: [[1.53341099]]
predict_vec: [0.00154882 0.0019864  0.00315657 0.00043608 0.00242976 0.00076895
 0.00335399 0.00505401 0.00150196 0.00034243 0.00015249 0.00148678
 0.07411966 0.08701551 0.06447499 0.07977743 0.11596128 0.12627125
 0.11144282 0.13662471]
cqt_chromas[similar_index]: [0.         0.00404752 0.00430094 0.         0.         0.00695415
 

1/1 [==============================] - 0s 23ms/step
score: [[1.53545721]]
predict_vec: [0.00277989 0.00102067 0.00032312 0.00377879 0.00098834 0.00067732
 0.0019965  0.00207654 0.00074878 0.00051635 0.00061525 0.0075634
 0.02928418 0.04743365 0.03385203 0.04040705 0.03452965 0.02925293
 0.02877924 0.03321714]
cqt_chromas[similar_index]: [0.         0.         0.         0.0078125  0.         0.0047091
 0.00158977 0.00242524 0.         0.         0.00231737 0.
 0.0779471  0.12504883 0.12317812 0.09030421 0.05952272 0.11894163
 0.11799628 0.12070578]
similar_index: 39336
--
-- generate 18 / 30
1/1 [==============================] - 0s 25ms/step
score: [[1.52702916]]
predict_vec: [0.00027286 0.00037968 0.00024003 0.00350253 0.00426833 0.00448489
 0.00133951 0.0004779  0.00191315 0.00104098 0.00612275 0.00228551
 0.06659929 0.05058893 0.02977157 0.06617572 0.062308   0.06936301
 0.067775   0.06217188]
cqt_chromas[similar_index]: [0.0078125  0.         0.         0.         0.00282375 0.002

1/1 [==============================] - 0s 19ms/step
score: [[1.54570034]]
predict_vec: [0.00195923 0.00407587 0.00674303 0.0020186  0.00224083 0.0006874
 0.00188181 0.00106342 0.00041103 0.00106841 0.00117014 0.00050031
 0.05969583 0.05195685 0.07305799 0.1265856  0.08501469 0.09848676
 0.08032148 0.08429025]
cqt_chromas[similar_index]: [0.         0.00543852 0.00510552 0.         0.00328379 0.00359226
 0.         0.         0.00510198 0.00490802 0.0026925  0.0078125
 0.11442801 0.09658157 0.09361717 0.11999644 0.1494688  0.16328327
 0.14095707 0.15293379]
similar_index: 7161
--
-- generate 23 / 30
1/1 [==============================] - 0s 18ms/step
score: [[1.52033527]]
predict_vec: [0.00202544 0.00243532 0.00469216 0.00445696 0.0032127  0.00037606
 0.00153166 0.00635641 0.00147035 0.00423767 0.00050448 0.00049658
 0.05129528 0.06670976 0.07710669 0.09112391 0.07643572 0.08724348
 0.08372632 0.07878299]
cqt_chromas[similar_index]: [0.007264   0.         0.00668417 0.00501286 0.       

1/1 [==============================] - 0s 17ms/step
score: [[1.48172067]]
predict_vec: [0.00064402 0.00077181 0.00173733 0.00350531 0.00410398 0.00157074
 0.00263351 0.00062198 0.00103223 0.00217752 0.00037614 0.00260283
 0.03862335 0.05311665 0.03263254 0.05464714 0.05003462 0.02625705
 0.03331785 0.04045166]
cqt_chromas[similar_index]: [0.00644406 0.         0.00741332 0.0078125  0.00541006 0.
 0.         0.         0.         0.         0.         0.
 0.0457196  0.04238311 0.03271674 0.03310983 0.01868367 0.01367575
 0.0336003  0.04830659]
similar_index: 31489
--
-- generate 28 / 30
1/1 [==============================] - 0s 22ms/step
score: [[1.52304917]]
predict_vec: [8.14819941e-04 1.59709912e-03 1.90983672e-04 1.83438868e-04
 1.89863064e-03 1.78026990e-03 4.72293328e-03 9.48340108e-04
 4.75596753e-04 1.36366102e-03 1.07271690e-03 9.79044780e-05
 1.83504224e-02 1.80095211e-02 8.50324333e-03 8.28393362e-03
 1.73473340e-02 2.73747109e-02 2.84601208e-02 3.03191487e-02]
cqt_chromas[si